In [1]:
"""
Purpose: To pull down the latest segment ids that 
correspond to the nucleus table so they can be used to 
run downstream tables

"""

'\nPurpose: To pull down the latest segment ids that \ncorrespond to the nucleus table so they can be used to \nrun downstream tables\n\n'

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from os import sys
sys.path.append("/meshAfterParty/")

In [15]:
import datajoint_utils as du
nucleus_table = du.configure_nucleus_table()
nucleus_table

ver materialization version,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,segment_id id of the segment under the nucleus centroid. Equivalent to Allen 'pt_root_id'.,"nucleus_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",supervoxel_id id of the supervoxel under the nucleus centroid. Equivalent to Allen: 'pt_supervoxel_id'.,volume volume of the nucleus in um^3
30.00,996,0,60464,93616,20968,0,35.1478
30.00,1833,0,56800,97280,19929,0,35.934
30.00,1841,0,57536,105584,19883,0,265.586
30.00,1896,0,59872,96608,19853,0,174.558
30.00,1998,864691136050815731,59936,105872,20078,72978435697419638,137.669
30.00,2005,0,61472,96128,20003,0,71.6715
30.00,2020,0,62448,97920,20177,0,214.823
30.00,2026,0,60832,100752,19857,0,103.014
30.00,2051,0,61968,96704,20133,0,38.5804
30.00,2198,864691135084884330,65088,104160,20059,73752285724957558,76.0236


In [17]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-13 07:14:50,026 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-13 07:14:50,316 - settings - Setting enable_python_native_blobs to True


In [21]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30

key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                        "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") & (minnie.OldBaylorSegmentCentroid() & "multiplicity<3").proj()
                                                             & (dj.U("segment_id") & nucleus_table))
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691136050815731,0,0.25
864691136521572241,0,0.25
864691135494586958,0,0.25
864691136451074559,0,0.25
864691134684964653,0,0.25
864691134917390346,0,0.25
864691135777154237,0,0.25
864691135269802405,0,0.25
864691135724233643,0,0.25
864691136436395166,0,0.25


In [22]:
new_seg_ids = (dj.U("segment_id") & nucleus_table).fetch("segment_id")

In [24]:
filtered_seg_ids = new_seg_ids[new_seg_ids!= 0]

In [27]:
filtered_seg_ids_dict = [dict(segment_id=k) for k in filtered_seg_ids]

In [28]:
@schema
class NucleiSegmentsRun2(dj.Manual):
    definition="""
    segment_id : bigint unsigned  #segment id for those to be decimated
    """
    
NucleiSegmentsRun2.insert(filtered_seg_ids_dict,skip_duplicates=True)